In [1]:
import sys
import os
from os import path
# 部署的路径
sys.path.append('/root/bdrisk/risk_project')
# 堡垒机的路径
sys.path.append(path.dirname(path.dirname(path.dirname(os.getcwd()))))
# 这是log文件的存放路径
root_path = path.dirname(path.dirname(path.dirname(os.getcwd())))
from risk_models import *
# from risk_models import 
import datetime
import numpy as np

In [78]:
# org_code='91310000583489628X'
org_code ='9131000008793522XH'

In [79]:
sql='''
select title,case_date,corp_name ,cus_code,legal_rep,entry_num,
case  when case_type = 'zswg' then '走私违规'
      when case_type = 'jyjy' then '检验检疫'
      when case_type = 'zscq' then '知识产权' 
end as case_type
from ods_zmxpq.customs_credit where iscurrent =1
'''

In [80]:
df_news = Read_Oracle().read_oracle(sql=sql, database='dbods')

2021-06-24 16:15:19.404 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 2.125s


In [82]:
sql=f'''
select ORG_CODE,org_name from BD_RISK_CORP_INFO_BASIC where iscurrent =1 
and ORG_CODE = '{org_code}'
'''

In [83]:
df_corp = Read_Oracle().read_oracle(sql=sql, database='dbods')

2021-06-24 16:15:31.955 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.145s


In [85]:
df = df_news.merge(df_corp,left_on='CORP_NAME',right_on='ORG_NAME',how='inner')

In [62]:
df.drop(['CORP_NAME'],axis=1,inplace=True)
df.rename(columns={'TITLE':'CASE_TITLE'},inplace=True)

In [63]:
df['SCORE'] = -10

In [64]:
df.fillna('',inplace=True)

In [65]:
df['CHECK_TIME'] = datetime.datetime.strftime(datetime.datetime.now(), format='%Y-%m-%d %H:%M:%S')
df['CHECK_TIME'] = pd.to_datetime(df['CHECK_TIME'], format='%Y-%m-%d %H:%M:%S')

In [66]:
df = df.reset_index().rename(columns={'index': 'ID'})

In [67]:
df

,ID,CASE_TITLE,CASE_DATE,CUS_CODE,LEGAL_REP,ENTRY_NUM,CASE_TYPE,ORG_CODE,ORG_NAME,SCORE,CHECK_TIME
0,0,中华人民共和国洋山海关关于上海同景国际物流发展有限公司行政处罚决定书（沪洋山...,2020-04-13,,陈昕,220120191000406892,走私违规,91310000583489628X,上海同景国际物流发展有限公司,-10,2021-06-24 11:39:14


In [74]:
df['CASE_DATE'] = pd.to_datetime(df['CASE_DATE'], format='%Y-%m-%d')

In [76]:
Write_Oracle().write_oracle('BD_RISK_RESULT_TRADE_TD1', df, org_code=org_code,alarm=None)

2021-06-24 11:47:05.128 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.083s
2021-06-24 11:47:05.244 | INFO     | risk_models.config.write_config.write_func:write_oracle:181 - Processing... Writing 1 rows into database
2021-06-24 11:47:05.279 | INFO     | risk_models.config.write_config.write_func:write_oracle:188 - Insert data into BD_RISK_RESULT_TRADE_TD1 successfully! Total write time spent 0.241s


In [68]:
def get_num(case_type):
    try:
        return df[df['CASE_TYPE']==case_type].groupby(['CASE_TYPE'])['ID'].count().iloc[0]
    except:
        return  0

In [69]:
num1 = get_num('走私违规')
num2 = get_num('检验检疫')
num3 = get_num('知识产权')

In [70]:
alarm_reason = [{'case_type':'走私违规','num': num1},
                {'case_type':'检验检疫','num': num2},
                {'case_type':'知识产权','num': num3}
                ]
df_alarm = []
for i in alarm_reason:
    if i['num'] != 0:
        
        temp = {}
        temp['CHECK_TIME'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        temp['ORG_CODE'] = '91310000583489628X'
        temp['MODEL_CODE'] = 'TRADE'
        temp['CHILD_MODEL_CODE'] = 'TD1'
        temp['ALARM_REASON'] = f'发现涉及'+i['case_type']+'案件'+str(i['num'])+'起'
        temp['ALARM_NUMBER'] = int(i['num']) 
        df_alarm.append(temp)
    else:
        pass
df_alarm = pd.DataFrame(df_alarm)
df_alarm = df_alarm.reset_index().rename(columns={'index': 'ID'})
df_alarm['CHECK_TIME'] = pd.to_datetime(df_alarm['CHECK_TIME'], format='%Y-%m-%d %H:%M:%S')
df_alarm

,ID,CHECK_TIME,ORG_CODE,MODEL_CODE,CHILD_MODEL_CODE,ALARM_REASON,ALARM_NUMBER
0,0,2021-06-24 11:39:17,91310000583489628X,TRADE,TD1,发现涉及走私违规案件1起,1


In [77]:
if df_alarm.empty:
    print('没有异常情况')
else:
    Write_Oracle().write_oracle('BD_RISK_ALARM_ITEM', df_alarm, org_code=org_code,
                                alarm=['TRADE', 'TD1'])

2021-06-24 11:47:43.356 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 3.692s
2021-06-24 11:47:43.546 | INFO     | risk_models.config.write_config.write_func:write_oracle:181 - Processing... Writing 1 rows into database
2021-06-24 11:47:43.598 | INFO     | risk_models.config.write_config.write_func:write_oracle:188 - Insert data into BD_RISK_ALARM_ITEM successfully! Total write time spent 3.940s
